In [1]:
print('OK!')

OK!


In [11]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [3]:
PINECONE_API_KEY="3fdb7357-cd42-4cc5-bbf3-1bd862bec155"
PINECONE_API_ENV="quickstart"

In [12]:
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [13]:
extracted_data=load_pdf("data/")

In [14]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [15]:
text_chunks = text_split(extracted_data)
print('Length of chunk: ',len(text_chunks))

Length of chunk:  7020


In [18]:
def download_hugging_face_model():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [19]:
embeddings = download_hugging_face_model()

f:\IIIT Bhubaneswar\ML\Medical Chatbot using Llama2\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` instead.
  warnings.warn(warning_message, FutureWarning)
f:\IIIT Bhubaneswar\ML\Medical Chatbot using Llama2\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [20]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [26]:
from pinecone import Pinecone

# Create an instance of Pinecone class
pinecone_instance = Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)

# Specify the index name and host
index_name = "medical-chatbot"
host = "https://medical-chatbot-w9g326t.svc.aped-4627-b74a.pinecone.io"  # Replace with your actual Pinecone host

# Create the index instance
index = pinecone_instance.Index(index_name=index_name, host=host)

# Assuming you have an embeddings model (HuggingFaceEmbeddings) and text_chunks ready
# Convert the texts into embeddings using the correct method (e.g., embed_documents)
vectors = embeddings.embed_documents([t.page_content for t in text_chunks])

# Prepare the data for upserting
# Each vector should be associated with a unique ID
ids = [f"id-{i}" for i in range(len(vectors))]
upsert_data = list(zip(ids, vectors))

# Batch size for each upsert (ensure each batch is small enough to fit the 4MB limit)
batch_size = 100  # Adjust batch size depending on the size of your vectors

# Function to upsert in batches
def upsert_in_batches(index, upsert_data, batch_size):
    for i in range(0, len(upsert_data), batch_size):
        batch = upsert_data[i:i + batch_size]
        index.upsert(vectors=batch)

# Upsert the vectors into the Pinecone index in batches
upsert_in_batches(index, upsert_data, batch_size)


In [35]:
print(type(query_embedding))  # Should be a list or numpy array of floats

<class 'list'>


In [36]:
print(len(query_embedding))  # Length of query embedding
# Assuming 'vectors' contains the document embeddings:
print(len(vectors[0]))  # Length of the document embeddings in the index


384
384


In [ ]:
import pinecone
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

# Create a Pinecone client instance
pinecone_client = pinecone.Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)

# Index name
index_name = "medical-chatbot"

# Embeddings model (You can use Hugging Face model)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create index if it doesn't exist
if index_name not in pinecone_client.list_indexes():
    # Define the dimension of the embeddings
    embedding_dim = len(embeddings.embed_query("test"))
    
    # Create the index directly with the dimension and metric
    pinecone_client.create_index(
        name=index_name,
        dimension=embedding_dim,  # The dimension of the embeddings
        metric='cosine'           # Metric for similarity (cosine similarity)
    )

# Connect to the index
index = pinecone_client.Index(index_name)

# Assuming you have text_chunks available
texts = [t.page_content for t in text_chunks]

# Convert the texts into embeddings
vectors = embeddings.embed_documents(texts)

# Generate unique IDs for the text chunks
ids = [f"id-{i}" for i in range(len(vectors))]

# Prepare upsert data
upsert_data = list(zip(ids, vectors))

# Batch size for upserting
batch_size = 100

# Function to upsert the vectors in batches
def upsert_in_batches(index, upsert_data, batch_size):
    for i in range(0, len(upsert_data), batch_size):
        batch = upsert_data[i:i + batch_size]
        index.upsert(vectors=batch)

# Upsert the vectors into the Pinecone index
upsert_in_batches(index, upsert_data, batch_size)

# Now, let's perform a similarity search with a query
query = 'What are allergies?'

# Embed the query
query_embedding = embeddings.embed_query(query)

# Perform the similarity search with Pinecone
result = index.query(queries=[query_embedding], top_k=3)

# Print the result
print(result)

In [52]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [53]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [54]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])